In [2]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind
import time
import os
# from pandas_profiling import ProfileReport
# pd.set_option('display.max_columns', None)
from IPython.display import display, HTML
from sklearn.model_selection import train_test_split

In [3]:
file_path = "/data/projects/Tissue_specific_data/subsample/Data/all_input_stratified.feather"
#output_path = "/data/projects/Tissue_specific_data/Tissue_specific_transcript"

In [4]:
df= pd.read_feather(file_path)

In [5]:
df.shape

(17382, 163725)

In [6]:
df.head()

,sample_class,sample,ENST00000373020.8,ENST00000494424.1,ENST00000496771.5,ENST00000612152.4,ENST00000614008.4,ENST00000373031.4,ENST00000485971.1,ENST00000371582.8,...,ENST00000640157.1,ENST00000640623.1,ENST00000640630.1,ENST00000638486.1,ENST00000381568.9,ENST00000622217.1,ENST00000408734.1,ENST00000385204.1,ENST00000583027.1,ENST00000638236.1
0,Adipose_Tissue,GTEX-1117F-0226-SM-5GZZ7,26.32,0.0,2.53,0.47,0.23,8.73,2.84,0.91,...,0.0,0.0,0.0,0.0,0.00,0.48,0.0,0.0,0.0,0.0
1,Muscle,GTEX-1117F-0426-SM-5EGHI,3.95,0.0,0.48,0.21,0.24,0.44,0.15,1.73,...,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0
2,Blood_Vessel,GTEX-1117F-0526-SM-5EGHJ,13.23,0.0,1.98,0.15,1.32,1.49,0.26,0.00,...,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0
3,Blood_Vessel,GTEX-1117F-0626-SM-5N9CS,30.15,0.0,4.18,1.23,1.15,4.70,1.48,0.52,...,0.0,0.0,0.0,0.0,0.00,1.54,0.0,0.0,0.0,0.0
4,Heart,GTEX-1117F-0726-SM-5GIEN,6.60,0.0,0.38,0.31,0.74,0.03,0.00,0.00,...,0.1,0.0,0.0,0.0,0.07,0.68,0.0,0.0,0.0,0.0


In [7]:
df.groupby('sample_class').size()

sample_class
Adipose_Tissue     1204
Adrenal_Gland       258
Bladder              21
Blood               929
Blood_Vessel       1335
Brain              2642
Breast              459
Cervix_Uteri         19
Colon               779
Esophagus          1445
Fallopian_Tube        9
Heart               861
Kidney               89
Liver               226
Lung                578
Muscle              803
Nerve               619
Ovary               180
Pancreas            328
Pituitary           283
Prostate            245
Salivary_Gland      162
Skin               1809
Small_Intestine     187
Spleen              241
Stomach             359
Testis              361
Thyroid             653
Uterus              142
Vagina              156
dtype: int64

In [8]:
all_sheets_list = pd.read_excel('/data/projects/Tissue_specific_data/TissueEnhanced/TenhTrans.xlsx', sheet_name=None, engine='openpyxl').values()

In [9]:
for i in range(len(list(all_sheets_list))):
    display(list(all_sheets_list)[i])

,transcript,tissue
0,ENST00000001146.6,"Adipose_Tissue, Bladder, Blood_Vessel, Breast,..."
1,ENST00000003084.10,"Pancreas, Salivary_Gland, Testis"
2,ENST00000003583.12,"Brain, Pituitary, Testis"
3,ENST00000004531.14,"Nerve, Ovary"
4,ENST00000005180.8,"Adipose_Tissue, Adrenal_Gland, Cervix_Uteri, F..."
...,...,...
28078,ENST00000640924.1,"Cervix_Uteri, Nerve, Ovary, Uterus"
28079,ENST00000640939.1,"Adrenal_Gland, Ovary"
28080,ENST00000640961.1,"Brain, Nerve"
28081,ENST00000640964.1,"Testis, Thyroid"


,transcript,tissue
0,ENST00000001146.6,"Adipose_Tissue, Blood_Vessel, Breast, Nerve, P..."
1,ENST00000002596.5,"Cervix_Uteri, Esophagus, Fallopian_Tube, Nerve..."
2,ENST00000003084.10,"Pancreas, Salivary_Gland, Testis"
3,ENST00000003583.12,"Brain, Testis"
4,ENST00000005180.8,"Adipose_Tissue, Adrenal_Gland, Nerve, Ovary, P..."
...,...,...
24903,ENST00000640851.1,"Brain, Pituitary, Testis"
24904,ENST00000640900.1,"Heart, Liver, Lung, Muscle, Pituitary, Spleen"
24905,ENST00000640924.1,"Nerve, Uterus"
24906,ENST00000640955.1,"Adipose_Tissue, Adrenal_Gland, Breast, Kidney,..."


,transcript,tissue
0,ENST00000001146.6,"Adipose_Tissue, Blood_Vessel, Breast, Fallopia..."
1,ENST00000003084.10,"Pancreas, Salivary_Gland, Small_Intestine, Testis"
2,ENST00000003583.12,"Brain, Pituitary, Testis"
3,ENST00000004531.14,"Nerve, Ovary"
4,ENST00000005180.8,"Adipose_Tissue, Adrenal_Gland, Cervix_Uteri, F..."
...,...,...
29159,ENST00000640939.1,"Adrenal_Gland, Ovary"
29160,ENST00000640961.1,"Brain, Fallopian_Tube, Nerve"
29161,ENST00000640964.1,"Testis, Thyroid"
29162,ENST00000640979.1,"Bladder, Fallopian_Tube, Prostate, Testis"


,transcript,tissue
0,ENST00000001146.6,"Adipose_Tissue, Blood_Vessel, Breast, Nerve, P..."
1,ENST00000002596.5,"Cervix_Uteri, Fallopian_Tube, Nerve, Ovary, Pi..."
2,ENST00000003084.10,"Pancreas, Salivary_Gland, Testis"
3,ENST00000003583.12,"Brain, Testis"
4,ENST00000005180.8,"Adipose_Tissue, Adrenal_Gland, Nerve, Ovary, P..."
...,...,...
25556,ENST00000640851.1,"Brain, Pituitary, Testis"
25557,ENST00000640900.1,"Heart, Lung, Muscle, Pituitary"
25558,ENST00000640955.1,"Adipose_Tissue, Adrenal_Gland, Bladder, Breast..."
25559,ENST00000640961.1,"Brain, Fallopian_Tube"


,transcript,tissue
0,ENST00000001146.6,"Adipose_Tissue, Blood_Vessel, Breast, Kidney, ..."
1,ENST00000003084.10,"Lung, Pancreas, Salivary_Gland, Small_Intestin..."
2,ENST00000003583.12,"Brain, Pituitary, Testis"
3,ENST00000005180.8,"Adipose_Tissue, Adrenal_Gland, Cervix_Uteri, K..."
4,ENST00000005558.8,"Nerve, Testis"
...,...,...
30127,ENST00000640958.1,"Bladder, Cervix_Uteri, Fallopian_Tube"
30128,ENST00000640961.1,"Brain, Fallopian_Tube, Nerve"
30129,ENST00000640969.1,"Muscle, Uterus"
30130,ENST00000640979.1,"Bladder, Fallopian_Tube, Testis"


,transcript,tissue
0,ENST00000001146.6,"Adipose_Tissue, Blood_Vessel, Breast, Nerve, P..."
1,ENST00000002596.5,"Cervix_Uteri, Fallopian_Tube, Nerve, Ovary, Pi..."
2,ENST00000003084.10,"Pancreas, Salivary_Gland, Testis"
3,ENST00000003583.12,"Brain, Testis"
4,ENST00000005180.8,"Adrenal_Gland, Cervix_Uteri, Nerve, Ovary, Pit..."
...,...,...
27544,ENST00000640939.1,"Adrenal_Gland, Ovary"
27545,ENST00000640955.1,"Adrenal_Gland, Bladder, Brain, Breast, Fallopi..."
27546,ENST00000640958.1,"Bladder, Fallopian_Tube"
27547,ENST00000640961.1,"Brain, Fallopian_Tube"


In [10]:
for sheet_no in range(4, len(list(all_sheets_list))):
    print("Sheet_No :", sheet_no)

Sheet_No : 4
Sheet_No : 5


In [ ]:
no_of_iteration =100
for sheet_no in range(5,len(list(all_sheets_list))):
    print("Sheet_No :", sheet_no)
    for index, row in list(all_sheets_list)[sheet_no].iterrows():
        transcript_id = row['transcript']
        tissue_enhanced_list = [e.strip() for e in row['tissue'].split(',')]
        #print(transcript_id, tissue_enhanced_list)
        df_temp = df[['sample_class','sample', transcript_id]]
        #print(df_temp.groupby(['sample_class']).size())
        if((index%1000)==0):
            print(index, transcript_id)
        index =index +1
        for iteration in range(no_of_iteration):
            sample_70_percent, _ = train_test_split(df_temp, test_size=0.3, stratify=df['sample_class'])
            #print(sample_70_percent.groupby(['sample_class']).size())
            #display(sample_70_percent)
            #list_1 = sample_70_percent[sample_70_percent[row['sample_class']].isin(row['tissue'].strip().split(','))][row['transcript']].tolist()
            #list_1 = sample_70_percent[sample_70_percent[row['sample_class']].isin(row['tissue'].strip().split(','))][row['transcript']].tolist()
            list_1 = sample_70_percent[sample_70_percent['sample_class'].isin(tissue_enhanced_list)][transcript_id].tolist()
            list_2 = sample_70_percent[~sample_70_percent['sample_class'].isin(tissue_enhanced_list)][transcript_id].tolist()
            #print(len(list_1), len(list_2))
            t, p = ttest_ind(list_1, list_2, equal_var=False)
            list(all_sheets_list)[sheet_no].loc[index, ['ITER_{}'.format(iteration)]] = p
        #display(list(all_sheets_list)[0])
    list(all_sheets_list)[sheet_no].to_csv("/data/projects/Tissue_specific_data/TissueEnhanced/Output/sheet_{}.tsv".format(sheet_no) ,sep="\t", index=False)

Sheet_No : 5
0 ENST00000001146.6
1000 ENST00000274625.5
2000 ENST00000318357.8
3000 ENST00000350881.6
4000 ENST00000369931.3
5000 ENST00000380851.9
6000 ENST00000398519.9
7000 ENST00000417154.5
8000 ENST00000431168.1


In [1]:
list(all_sheets_list)[0]

NameError: name 'all_sheets_list' is not defined

In [91]:
df_p_values

,Transcript_ID,ITER_0,ITER_1,ITER_2
0,ENST00000373020.8,0.000000e+00,0.000000e+00,0.000000e+00
1,ENST00000494424.1,2.815337e-52,2.946357e-51,7.525728e-51
2,ENST00000496771.5,0.000000e+00,0.000000e+00,0.000000e+00
3,ENST00000612152.4,0.000000e+00,0.000000e+00,0.000000e+00
4,ENST00000614008.4,6.386024e-99,6.654822e-99,8.867693e-98
...,...,...,...,...
163718,ENST00000622217.1,3.423964e-11,2.541612e-11,5.301768e-12
163719,ENST00000408734.1,2.206672e-09,2.416088e-09,8.935349e-13
163720,ENST00000385204.1,3.632762e-04,5.348432e-04,1.799940e-03
163721,ENST00000583027.1,6.600304e-02,3.693325e-02,1.126524e-01


In [92]:
df_p_values.set_index('Transcript_ID', inplace=True)
print (df_p_values)

                         ITER_0        ITER_1        ITER_2
Transcript_ID                                              
ENST00000373020.8  0.000000e+00  0.000000e+00  0.000000e+00
ENST00000494424.1  2.815337e-52  2.946357e-51  7.525728e-51
ENST00000496771.5  0.000000e+00  0.000000e+00  0.000000e+00
ENST00000612152.4  0.000000e+00  0.000000e+00  0.000000e+00
ENST00000614008.4  6.386024e-99  6.654822e-99  8.867693e-98
...                         ...           ...           ...
ENST00000622217.1  3.423964e-11  2.541612e-11  5.301768e-12
ENST00000408734.1  2.206672e-09  2.416088e-09  8.935349e-13
ENST00000385204.1  3.632762e-04  5.348432e-04  1.799940e-03
ENST00000583027.1  6.600304e-02  3.693325e-02  1.126524e-01
ENST00000638236.1  0.000000e+00  0.000000e+00  0.000000e+00

[163723 rows x 3 columns]


In [103]:
def count(df):
    #n=df.shape[1]
    #print(df)
    threshold_values=[0.05]
    for threshold in threshold_values:
        r = np.sum(df >= threshold)
        #print (r, df, no_of_iteration)
        df['>={}'.format(threshold)] = (r+1) /(no_of_iteration+1)
    #df['r'] = np.sum(df == 'r')
    return df

In [104]:
start_time = time.time()
temp_df_empirical = df_p_values.apply(count, axis = 1)
print("--- %s seconds ---" % (time.time() - start_time))

--- 66.26727819442749 seconds ---


In [105]:
temp_df_empirical

,ITER_0,ITER_1,ITER_2,>=0.05
Transcript_ID,,,,
ENST00000373020.8,0.000000e+00,0.000000e+00,0.000000e+00,0.25
ENST00000494424.1,2.815337e-52,2.946357e-51,7.525728e-51,0.25
ENST00000496771.5,0.000000e+00,0.000000e+00,0.000000e+00,0.25
ENST00000612152.4,0.000000e+00,0.000000e+00,0.000000e+00,0.25
ENST00000614008.4,6.386024e-99,6.654822e-99,8.867693e-98,0.25
...,...,...,...,...
ENST00000622217.1,3.423964e-11,2.541612e-11,5.301768e-12,0.25
ENST00000408734.1,2.206672e-09,2.416088e-09,8.935349e-13,0.25
ENST00000385204.1,3.632762e-04,5.348432e-04,1.799940e-03,0.25


In [34]:
df_mean = df.groupby(tissue_name).mean(numeric_only=True)
index_list = list(df_mean.index.values)

,index,sample_class,sample,ENST00000373020.8,ENST00000494424.1,ENST00000496771.5,ENST00000612152.4,ENST00000614008.4,ENST00000373031.4,ENST00000485971.1,...,ENST00000640157.1,ENST00000640623.1,ENST00000640630.1,ENST00000638486.1,ENST00000381568.9,ENST00000622217.1,ENST00000408734.1,ENST00000385204.1,ENST00000583027.1,ENST00000638236.1
0,10615,Adipose_Tissue,GTEX-1K9T9-0226-SM-E9U6K,23.12,0.0,0.52,0.98,0.61,44.83,7.91,...,0.0,0.00,0.0,0.1,0.00,0.00,0.0,0.0,0.0,0.00
1,8241,Adipose_Tissue,GTEX-1GMRU-0226-SM-9YFLW,27.44,0.0,2.31,1.29,0.97,46.13,12.50,...,0.0,0.00,0.0,0.0,0.00,0.12,0.0,0.0,0.0,0.15
2,9442,Adipose_Tissue,GTEX-1I1CD-1026-SM-COH3G,36.05,0.0,3.80,1.29,0.50,4.45,2.68,...,0.0,0.00,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.05
3,11009,Adipose_Tissue,GTEX-1LSNL-0526-SM-DIPFI,18.77,0.0,0.96,1.09,0.56,17.69,2.36,...,0.0,0.00,0.0,0.0,0.00,1.14,0.0,0.0,0.0,0.09
4,15630,Adipose_Tissue,GTEX-XV7Q-1726-SM-4BRUU,23.23,0.0,2.41,1.39,0.07,1.20,0.29,...,0.0,0.00,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13156,17169,Vagina,GTEX-ZXES-0626-SM-5E45P,26.12,0.0,1.15,0.77,0.65,0.06,0.00,...,0.0,0.02,0.0,0.0,0.00,0.49,0.0,0.0,0.0,0.40
13157,6377,Vagina,GTEX-18A7A-2826-SM-7LT8J,25.81,0.0,1.41,0.63,0.00,0.25,0.21,...,0.0,0.00,0.0,0.0,0.00,1.80,0.0,0.0,0.0,0.00
13158,2603,Vagina,GTEX-13N11-1526-SM-5L3FL,24.88,0.0,8.22,2.97,2.48,0.11,0.28,...,0.0,0.00,0.0,0.0,0.02,0.36,0.0,0.0,0.0,1.38
13159,258,Vagina,GTEX-11DXX-1726-SM-5H11T,24.70,0.0,1.95,0.76,1.83,0.24,0.00,...,0.0,0.00,0.0,0.0,0.29,0.00,0.0,0.0,0.0,0.25


In [19]:
temp_df.groupby(['sample_class']).size()

sample_class
Breast    459
dtype: int64

In [20]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Sample dataframe
data = {
    'ID': list(range(1, 201)),
    'sample_name': ['Tissue_' + str(i%10 + 1) for i in range(200)]  # 10 types of tissues
}
df = pd.DataFrame(data)

# Use train_test_split to stratify and get 70% samples from each tissue type
sample_70_percent, _ = train_test_split(df, test_size=0.3, stratify=df['sample_name'], random_state=42)

print(sample_70_percent)


      ID sample_name
199  200   Tissue_10
174  175    Tissue_5
96    97    Tissue_7
14    15    Tissue_5
3      4    Tissue_4
..   ...         ...
177  178    Tissue_8
175  176    Tissue_6
97    98    Tissue_8
103  104    Tissue_4
122  123    Tissue_3

[140 rows x 2 columns]


In [21]:
df.groupby(['sample_name']).size()

sample_name
Tissue_1     20
Tissue_10    20
Tissue_2     20
Tissue_3     20
Tissue_4     20
Tissue_5     20
Tissue_6     20
Tissue_7     20
Tissue_8     20
Tissue_9     20
dtype: int64

In [22]:
sample_70_percent.groupby(['sample_name']).size()

sample_name
Tissue_1     14
Tissue_10    14
Tissue_2     14
Tissue_3     14
Tissue_4     14
Tissue_5     14
Tissue_6     14
Tissue_7     14
Tissue_8     14
Tissue_9     14
dtype: int64